# Interleaved Randomized Benchmarking

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

**Interleaved Randomized Benchmarking (IRB)** is the algorithm for characterizing the specific individual Clifford gate by using randomization techniques, which is similar to the Standard Randomized Benchmarking (SRB).
This tutorial introduces **IRB**, covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).


## Theory

We will assume *a priori* that the readers are familiar with Clifford gates, or you can checkout the SRB module. We treat IRB as a modification to the SRB protocol, where we construct the random circuits using Clifford gates in SRB.


### Random circuits

The structure of random circuit in SRB is as followed:

![random-clifford-circuit](figures/srb-clifford-circuit.png "Figure 1: random Clifford circuit of SRB")

For experimental implementation of the random circuit above, we first apply $m$ random Clifford gates, and then we apply the inverse of whole $m$ Clifford gates $C_{m+1}$

$$
C_{m+1} := \left(C_1C_2\cdots C_{m-1}C_{m}\right)^{-1}.
$$

In the ideal situation, the output state should be equal to the input state

$$
C_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert \phi \rangle = \vert \phi \rangle.
$$

However, the structure of random circuit in IRB is quite different:

![random-clifford-circuit](figures/irb-clifford-circuit.png "Figure 2: random Clifford circuit of IRB")

In this figure, $C_i, i=1,2,\cdots,m$ are sequence of $m$ random Clifford gates, analogously to the above. $C$ is the gate of our interest that we wish to characterize, which is "interleaved" throughout the SRB sequence. Similarly, we apply the $C_{m+1}$ gate, which is inverse of whole $m$ Clifford gates $CC_i$.

### Randomized benchmarking

We can clearly see that the protocol is a simple extension of SRB. The exact process is as follows:

1. Run SRB on the qubits and estimate an average gate fidelity (AGF) $f$ of Clifford gates $C_i$.

2. Run IRB on the qubits and estimate an average gate fidelity $f_c$ of Clifford gates $CC_i$.

3. Use two fidelity parameters $f$ and $f_c$ to calculate the average error rate $r$ of gate $C$.

The average error rate $r$ must lie in the range $\left[r^{est}-E,r^{est}+E\right]$[1], where $r^{est}$ and $E$ are shown as follows, respectively.

$$
r^{est} = \frac{d-1}{d}\left(1-\frac{f_c}{f}\right),
$$


$$
E = min\left\{\frac{(d-1)\left[\vert f-f_c/f\vert + (1-f)\right]}{d},\frac{2(d^2-1)(1-f)}{fd^2} + \frac{4\sqrt{1-f}\sqrt{d^2-1}}{f}\right\}.
$$

## Practice
### Single qubit circuit

![single-qubit](figures/irb-single-clifford-circuit.png "Figure 3: single qubit circuit")

In the following code, we set the target quantum gate to the `H` gate. Please refer to the `Clifford` class or [2] for how Clifford gates are randomly generated. Furthermore, we construct and measure $20$ random circuits in each depth and average the results. Run the example and see the final result.

In [ ]:
import QCompute
from qcompute_qep.quantum.clifford import Clifford
from qcompute_qep.benchmarking.interleavedrb import InterleavedRB

# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"
qc = QCompute.BackendName.CloudBaiduQPUQian

# 1 qubit IRB
single_qubits = [0]
# generate a specific Clifford gate H gate
pattern = [[['A2'],[],['C1'],[],['E1']]]
target_gate = Clifford(1, pattern)
irb = InterleavedRB()
single_qubit_results = irb.benchmark(qubits=single_qubits,
                                     target_gate = target_gate,
                                     qc=qc,
                                     repeats=20,
                                     shots=1024,
                                     seq_lengths=[1, 5, 10, 15, 20, 25])
irb.plot_results()

### Two qubits circuit

The circuit is shown as follows

![double-qubit](figures/irb-two-clifford-circuit.png "Figure 4: two-qubit circuit")

In [ ]:
# 2 qubits IRB
two_qubits = [0,1]
# random generate an target two-qubit gate
target_gate = Clifford(2)
irb2 = InterleavedRB()
two_qubits_results = irb2.benchmark(qubits=two_qubits,
                                   target_gate=target_gate,
                                   qc=qc,
                                   repeats=20,
                                   shots=1024,
                                   seq_lengths=[1, 2, 3, 4, 5, 6])
irb2.plot_results()

As shown above, we obtain the curve and the parameters $f$, $f_c$, $r$ from fitting procedure.

We can print the result of parameters.

In [ ]:
print('The fidelity of 1 qubit SRB is :',single_qubit_results['StandardRB']['f'])
print('The fidelity of 1 qubit IRB is :',single_qubit_results['InterleavedRB']['f'])
print('The error rate of target gate is :',single_qubit_results['target_gate']['r'])
print('The fidelity of 2 qubit SRB is :',two_qubits_results['StandardRB']['f'])
print('The fidelity of 2 qubit IRB is :',two_qubits_results['InterleavedRB']['f'])
print('The error rate of target gate is :',two_qubits_results['target_gate']['r'])

From the single-qubit result, we can see that the fidelity of `H` gate from IRB implementation is different from the SRB implementation, which is more accurate. We have finished **IRB** and successfully obtained the error rate of a target quantum gate. More information of randomized benchmarking can be found in [3].

## References

\[1\] Magesan, Easwar, et al. "Efficient measurement of quantum gate error by interleaved randomized benchmarking." [Physical Review Letters](https://arxiv.org/abs/1203.4550) 109.8 (2012): 080505.

\[2\] Selinger, Peter. "Generators and relations for n-qubit Clifford operators." [Logical Methods in Computer Science](https://arxiv.org/abs/1310.6813v4) 11 (2015).

\[3\] Helsen, Jonas, et al. "A general framework for randomized benchmarking." [arXiv](https://arxiv.org/abs/2010.07974) preprint arXiv:2010.07974 (2020).